In [1]:
import os 

In [3]:
os.chdir('../')

In [4]:
pwd

'd:\\Electricity-Consumption-Prediction'

In [12]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen = True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    params_learning_rate : float
    input_shape: List[int]

In [14]:
from electrycityconsumption.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from electrycityconsumption.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_learning_rate=float(self.params.LEARNING_RATE),
            input_shape=self.params.input_shape
        )

        return prepare_base_model_config

In [38]:
import tensorflow as tf
from pathlib import Path


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def create_lstm_model(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.LSTM(50, input_shape=tuple(self.config.input_shape), return_sequences=True))
        model.add(tf.keras.layers.LSTM(50, return_sequences=True))
        model.add(tf.keras.layers.Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        self.save_model(path=self.config.base_model_path, model=model)
    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)


In [39]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.create_lstm_model()
except Exception as e:
    raise e

2023-10-24 16:00:09,408: INFO : common :yaml file: electrycityconsumption\config\config.yaml loaded successfully
2023-10-24 16:00:09,426: INFO : common :yaml file: electrycityconsumption\params.yaml loaded successfully
2023-10-24 16:00:09,433: INFO : common :created directory at: artifacts


c:\Users\Chetan Patel\anaconda3\envs\pyspark_env\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
